In [77]:
import numpy as np
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols

In [12]:
conn = pymysql.connect(
    host="192.168.7.93",
    user="quantchina",
    password="zMxq7VNYJljTFIQ8",
    database="wind",
    charset="gbk"
)
# 读取个股的换手率以及市值
cursor = conn.cursor()
query = """
    SELECT TRADE_DT, S_INFO_WINDCODE, S_DQ_TURN, S_VAL_MV
    FROM ASHAREEODDERIVATIVEINDICATOR
    WHERE TRADE_DT in (select TRADE_DT from AINDEXEODPRICES where S_INFO_WINDCODE='000985.CSI' and 
        TRADE_DT between '20230101' and '20240201') and 
        S_INFO_WINDCODE in (
            SELECT S_CON_WINDCODE
            FROM AINDEXCSIALLINDWEIGHT
            where TRADE_DT='20230104'
            ORDER BY S_CON_WINDCODE
    )
    ORDER BY TRADE_DT, S_INFO_WINDCODE
"""
cursor.execute(query)

data = cursor.fetchall()
df = pd.DataFrame(data, columns=['date', 'code', 'turnover', 'vals'])
df['turnover'] = df['turnover'].astype(float) / 100
df.head()

,date,code,turnover,vals
0,20230103,000001.SZ,0.011307,26721949.3586
1,20230103,000002.SZ,0.006549,21202783.3656
2,20230103,000006.SZ,0.078999,826196.9682
3,20230103,000007.SZ,0.032314,279930.0196
4,20230103,000008.SZ,0.006876,656267.7017


In [45]:
# 读取个股的行业信息
query = """
    SELECT WIND_CODE, WIND_IND_CODE
    FROM ASHAREINDUSTRIESCLASS
    WHERE CUR_SIGN=1 and WIND_CODE in (
        SELECT S_CON_WINDCODE
        FROM AINDEXCSIALLINDWEIGHT
        where TRADE_DT='20230104'
    )
    ORDER BY WIND_CODE
"""
cursor.execute(query)

ind_data = cursor.fetchall()
ind = pd.DataFrame(ind_data, columns=['code', 'industry'])
ind.set_index('code', inplace=True)
for i in range(len(ind)):
    ind.iloc[i, 0] = (ind.iloc[i, 0])[:4]

In [140]:
# 读取个股的收益率数据
query = """
    SELECT TRADE_DT, S_INFO_WINDCODE, S_DQ_PCTCHANGE
    FROM ASHAREEODPRICES    
    WHERE S_INFO_WINDCODE in (
        SELECT S_CON_WINDCODE
        FROM AINDEXCSIALLINDWEIGHT
        WHERE TRADE_DT='20230104'
    ) and TRADE_DT between '20230101' and '20240201'
    ORDER BY TRADE_DT, S_INFO_WINDCODE 
"""
cursor.execute(query)
data = cursor.fetchall()

In [141]:
returns = pd.DataFrame(data, columns=['date', 'code', 'pct_change'])
returns['pct_change'] = returns['pct_change'].astype(float) / 100

In [145]:
returns['pct_change']

0          0.046353
1          0.001648
2         -0.028571
3          0.038560
4          0.026087
             ...   
1190859   -0.013263
1190860   -0.033898
1190861   -0.010940
1190862   -0.014091
1190863    0.023993
Name: pct_change, Length: 1190864, dtype: float64

In [147]:
merge = pd.concat([df, returns['pct_change']], axis=1)
merge.dropna(inplace=True)

In [154]:
merge.shape

(1190864, 5)

In [174]:
factor_r = []
for date in tqdm(df.date.unique(), desc="Processing"):
    scaler = StandardScaler()
    
    df_ols = (merge[merge.date==date]).iloc[:, 1:]
    df_ols.set_index('code', inplace=True)
    df_ols = pd.merge(df_ols, ind, left_index=True, right_index=True, how='inner')
    df_ols.vals = scaler.fit_transform(df_ols.vals.to_numpy().reshape(-1, 1))
    
    dummies = pd.get_dummies(df_ols.industry)
    dummies.drop(['6260'], axis=1, inplace=True)
    df_dummies = pd.concat([df_ols, dummies], axis=1)
    y = df_dummies.turnover.to_numpy()
    X = sm.add_constant(df_dummies.drop(['turnover', 'industry', 'pct_change'], axis=1).to_numpy())
    results = sm.OLS(y, X).fit()
    resid = results.resid
    
    # 收益率对中性化后的换手率因子回归
    r = df_dummies['pct_change'].to_numpy()
    res = sm.OLS(r, resid).fit()
    factor_r.append(res.params[0])
    
    factor = resid.sort()
    

Processing: 100%|██████████| 265/265 [00:54<00:00,  4.88it/s]


In [ ]:
factor = res

In [175]:
aa = np.array([5, 3, 2, 1, 4])
aa.sort()
aa

array([1, 2, 3, 4, 5])

In [120]:
ind.loc['002013.SZ', ]

industry    6220
Name: 002013.SZ, dtype: object

In [163]:
resid

array([ 0.00912345, -0.0058865 ,  0.05959178, ..., -0.00164247,
       -0.0138255 , -0.02295757])

In [169]:
df_dummies['pct_change']

code
000001.SZ    0.046353
000002.SZ    0.001648
000006.SZ   -0.028571
000007.SZ    0.038560
000008.SZ    0.026087
               ...   
688799.SH    0.099277
688800.SH    0.012373
688819.SH    0.027490
688981.SH    0.009723
689009.SH    0.004264
Name: pct_change, Length: 4500, dtype: float64

In [166]:
r = df_dummies.pct_change.to_numpy()
r

AttributeError: 'function' object has no attribute 'to_numpy'

In [119]:
df_ols.iloc[4490:, :]

,turnover,vals,industry
code,,,
688788.SH,0.011402,-0.188695,6245
688789.SH,0.010674,-0.062648,6220
688793.SH,0.014465,-0.208753,6235
688798.SH,0.010330,-0.006576,6245
688799.SH,0.014669,-0.206483,6235
688800.SH,0.025322,-0.131052,6245
688819.SH,0.015176,0.160270,6220
688981.SH,0.071590,4.979680,6245
689009.SH,0.007315,0.043948,6225


In [64]:
dfols = (df[df.date=='20230103']).iloc[:, 1:]
dfols.set_index('code', inplace=True)
dfols = pd.concat([dfols, ind], axis=1)

dfols.head()

,turnover,vals,industry
code,,,
000001.SZ,0.011307,26721949.3586,6240
000002.SZ,0.006549,21202783.3656,6260
000006.SZ,0.078999,826196.9682,6260
000007.SZ,0.032314,279930.0196,6225
000008.SZ,0.006876,656267.7017,6220


In [51]:
df[df.date=='20230103'].turnover

0       0.011307
1       0.006549
2       0.078999
3       0.032314
4       0.006876
          ...   
4495    0.090426
4496    0.026200
4497    0.017400
4498    0.004524
4499    0.007394
Name: turnover, Length: 4500, dtype: float64

In [160]:
pd.read_parquet("/nas92/factor/stk_000985.CSI_20130101_20240202.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.